
# Freeze-in CFD — Demo Notebook

This notebook demonstrates the **ODE benchmark** and the **CFD-style finite-volume PDE** solver
for the freeze-in yield. It reproduces the main figure in the repository.

- Variable: $x \equiv m_\chi/T$
- Yield: $Y \equiv n_\chi/s$
- Reference ODE: $\tfrac{dY}{dx} = \tfrac{1}{s(T)}\,\tfrac{C(T)}{H(T)\,x}$


In [ ]:

# Setup: add local 'src' to Python path (to run from a cloned repo without installation)
import os, sys, numpy as np, matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 120

REPO_ROOT = os.getcwd()  # run this notebook from the repo root
SRC = os.path.join(REPO_ROOT, "src")
if SRC not in sys.path:
    sys.path.insert(0, SRC)

from freezein_fv.ivp_reference import solve_reference
from freezein_fv.pde_solver import march_pde


## Run ODE benchmark and PDE solver

In [ ]:

# ODE reference
ref = solve_reference()
x_ref, Y_ref = ref["x_plot"], ref["Y"]

# PDE (fixed-step in x, MUSCL in p), with an ODE→PDE switch at small x
out = march_pde(
    Np=600, Nx=2500, inj_width=0.02,
    scheme="muscl", time_integrator="rk2",
    x_switch=1e-2
)
x_pde   = out["x"]
Y_src   = out["Y_src_hist"]
Y_fromf = out["Y_hist"]

print("Final Y (ODE)         :", Y_ref[-1])
print("Final Y (PDE src-only):", Y_src[-1])
print("Final Y (integrated f):", Y_fromf[-1])


## Plot

In [ ]:

fig = plt.figure(figsize=(8.2, 4.8))
ax = plt.gca()

ax.loglog(x_ref, Y_ref, lw=2.0, label="ODE (solve_ivp)", color="C0")
ax.loglog(x_pde, Y_src, "--", lw=2.0, label="PDE (source-only)", color="C1")
ax.loglog(x_pde, Y_fromf, ":", lw=2.0, label="PDE (integrated f)", color="C2")

x_switch = 1e-2
ax.axvline(x_switch, color="C7", ls="--", lw=1.0)
# annotate near the switch
import numpy as _np
idx = _np.searchsorted(x_ref, x_switch, side="left")
y_here = Y_ref[min(idx, len(Y_ref)-1)]
ax.text(x_switch*1.05, y_here*1.3, "ODE → PDE\ntransition", fontsize=9, va="center")

ax.set_xlabel(r"$x = m_\chi/T$", fontsize=12)
ax.set_ylabel(r"$Y = n_\chi/s$", fontsize=12)
ax.set_title("Freeze-in yield comparison", fontsize=13)

ax.legend(frameon=False, fontsize=10, loc="lower right")
ax.grid(False)
ax.set_facecolor("white")
ax.tick_params(which="both", direction="in", top=True, right=True)
fig.tight_layout()
plt.show()


## Momentum-space snapshots

In [ ]:

# Re-run with snapshots at selected x values and plot p^2 f(p)
snaps = [1e-2, 3e-2, 1e-1, 3e-1, 1.0]
out2 = march_pde(Np=600, Nx=2500, inj_width=0.02, scheme='muscl',
                 time_integrator='rk2', x_switch=1e-2, snapshots_x=snaps)

items = sorted(out2['snapshots'].items(), key=lambda kv: kv[1]['x'])
from freezein_fv.physics import mA
plt.figure(figsize=(8.2, 4.8))
ax = plt.gca()
for req_x, data in items:
    p, f, x_act = data['p'], data['f'], data['x']
    ax.loglog(p, p**2*f, lw=2, label=rf"$x pprox {x_act:.2e}$")
ax.axvline(0.5*mA, ls="--", lw=1.2, color="C0")
ax.set_xlabel(r"$p\,[\mathrm{GeV}]$", fontsize=12)
ax.set_ylabel(r"$p^2 f_\chi(p)$ (arb.)", fontsize=12)
ax.set_title(r"Momentum-space snapshots of $f_\chi(p)$", fontsize=13)
ax.legend(frameon=False, fontsize=9)
ax.grid(False)
ax.set_facecolor("white")
ax.tick_params(which="both", direction="in", top=True, right=True)
plt.tight_layout()
plt.show()
